In [2]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

### Config

In [3]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20250522'

In [4]:
# pro_file_name = 'ACO Builder MSSP Prospective Explorer v2026.3c.txt'
# retro_file_name = 'ACO Builder MSSP Retrospective Explorer v2026.3c.txt'

pro_risk_score_file_name = 'ACO Builder MSSP Prospective Explorer v2026.3c - 2025 Risk Scores.txt'
retro_risk_score_file_name = 'ACO Builder MSSP Retrospective Explorer v2026.3c - 2025 Risk Scores.txt'

config = [
    # (pro_score_file_name, f'milliman_pro_{current_folder}'),
    # (retro_score_file_name, f'milliman_ret_{current_folder}'),
    
    (pro_risk_score_file_name, f'milliman_pro_risk_{current_folder}'),
    (retro_risk_score_file_name, f'milliman_ret_risk_{current_folder}'),
]

In [5]:
conn = cb_utils.get_engine(source='msh_analytics')
querries = []
for (file_name, table_name) in config:
    df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
    df = cb_utils.df_format_columns(df)
    print(f'{df.shape[0]} rows, {df.shape[1]} columns')
    display(df.head())
    
    schema = 'raw'
    print(f'select * from {schema}.{table_name}')
    querries.append(f'select * from {schema}.{table_name}')
    
    df.to_sql(table_name, conn, schema='raw', index=False, method='multi', chunksize=1000, if_exists='replace') # if_exists='replace', 

72467 rows, 46 columns


,pac_id,deid_tin,tin_public,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28,reg_ret_rs_esrd_legacy,reg_ret_rs_dis_legacy,reg_ret_rs_ad_legacy,reg_ret_rs_and_legacy,reg_ret_rs_comp_legacy,reg_pro_rs_esrd_legacy,reg_pro_rs_dis_legacy,reg_pro_rs_ad_legacy,reg_pro_rs_and_legacy,reg_pro_rs_comp_legacy,reg_ret_rs_esrd_v24,reg_ret_rs_dis_v28,reg_ret_rs_ad_v28,reg_ret_rs_and_v28,reg_ret_rs_comp_v28,reg_pro_rs_esrd_v24,reg_pro_rs_dis_v28,reg_pro_rs_ad_v28,reg_pro_rs_and_v28,reg_pro_rs_comp_v28
0,9335446202,00004B7C4023,474454561,2025,Pro,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404.0,0.0,8.00,1.15,74.0,83.15,NaN,1.298024,0.764234,0.915905,NaN,1.443064,0.704274,0.921253,NaN,0.945602,0.870972,0.982958,0.970026,NaN,0.975521,0.898748,1.013546,1.000714,NaN,0.931999,0.887232,1.014300,0.995237,NaN,0.967949,0.905247,1.042311,1.023560
1,5294898292,00019072C86C,263692916,2025,Pro,CHICAGO SPORTS AND SPINE LLC,NaN,NaN,Illinois,16984.0,IL-16984,"Chicago-Naperville-Evanston, IL",60605.0,0.0,1.18,1.15,18.0,20.33,NaN,0.434621,3.158932,0.960518,NaN,0.449224,3.050175,0.815552,NaN,1.018791,1.047385,1.017910,1.022423,NaN,1.040399,1.089200,1.055633,1.058236,NaN,1.035108,1.036543,1.024666,1.026372,NaN,1.047251,1.070751,1.057741,1.056105
2,Not Available,0005468C2905,330219886,2025,Pro,CHUNG BUI,Family Choice ACO LLC,NaN,California,11244.0,CA-11244,"Anaheim-Santa Ana-Irvine, CA",92683.0,0.0,0.00,85.00,1.2,86.20,NaN,NaN,0.871947,0.503288,NaN,NaN,0.859472,0.528512,NaN,NaN,0.947005,1.007318,0.949139,NaN,NaN,0.971131,1.046342,0.973786,NaN,NaN,0.919771,0.991150,0.922135,NaN,NaN,0.936901,1.023858,0.939784
3,446434757,00062442CB01,113492373,2025,Pro,ANDREW GOLDFARB MD PC,NaN,NaN,New York,35004.0,NY-35004,"Nassau County-Suffolk County, NY",11040.0,0.0,0.00,1.15,65.0,66.15,NaN,NaN,0.909085,0.841008,NaN,NaN,0.779431,0.880912,NaN,NaN,1.155471,0.995964,1.003062,NaN,NaN,1.191524,1.027416,1.032979,NaN,NaN,1.140698,1.010151,1.015099,NaN,NaN,1.170463,1.038197,1.041882
4,3971678558,0006DCCB459E,742797745,2025,Pro,WELLMED MEDICAL MANAGEMENT OF FLORIDA INC,WellMed Tampa/Orlando Medicare ACO LLC,NaN,Florida,38940.0,FL-38940,"Port St. Lucie, FL",34994.0,42.0,235.00,191.50,7324.5,7793.00,1.13338,1.086735,1.177661,1.261128,1.13338,0.969476,1.089179,1.169931,1.076763,1.147889,1.264855,1.199034,1.194663,1.108498,1.186246,1.316898,1.244636,1.239272,1.076763,1.122648,1.244180,1.123314,1.126696,1.108498,1.147523,1.286536,1.160205,1.162714


select * from raw.milliman_pro_risk_20250522
71311 rows, 46 columns


,pac_id,deid_tin,tin_public,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28,reg_ret_rs_esrd_legacy,reg_ret_rs_dis_legacy,reg_ret_rs_ad_legacy,reg_ret_rs_and_legacy,reg_ret_rs_comp_legacy,reg_pro_rs_esrd_legacy,reg_pro_rs_dis_legacy,reg_pro_rs_ad_legacy,reg_pro_rs_and_legacy,reg_pro_rs_comp_legacy,reg_ret_rs_esrd_v24,reg_ret_rs_dis_v28,reg_ret_rs_ad_v28,reg_ret_rs_and_v28,reg_ret_rs_comp_v28,reg_pro_rs_esrd_v24,reg_pro_rs_dis_v28,reg_pro_rs_ad_v28,reg_pro_rs_and_v28,reg_pro_rs_comp_v28
0,9335446202,00004B7C4023,474454561,2025,Retro,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404.0,0.0,9.00,1.14,72.00,82.14,NaN,1.065932,0.763104,1.129820,NaN,1.139311,0.650619,1.124145,NaN,0.953404,0.862109,0.995767,0.980005,NaN,0.0,0.0,0.0,NaN,NaN,0.948416,0.884028,1.026553,1.005878,NaN,0.0,0.0,0.0,NaN
1,5294898292,00019072C86C,263692916,2025,Retro,CHICAGO SPORTS AND SPINE LLC,NaN,NaN,Illinois,16984.0,IL-16984,"Chicago-Naperville-Evanston, IL",60605.0,0.0,1.18,1.14,15.00,17.32,NaN,1.385629,3.678623,0.767564,NaN,1.458054,2.869274,0.648044,NaN,1.014525,1.039464,1.013098,1.016487,NaN,0.0,0.0,0.0,NaN,NaN,1.031143,1.028566,1.020041,1.021356,NaN,0.0,0.0,0.0,NaN
2,Not Available,0005468C2905,330219886,2025,Retro,CHUNG BUI,Family Choice ACO LLC,NaN,California,11244.0,CA-11244,"Anaheim-Santa Ana-Irvine, CA",92683.0,0.0,0.00,40.00,1.21,41.21,NaN,NaN,0.886240,0.535467,NaN,NaN,0.913328,0.502371,NaN,NaN,0.946824,1.006657,0.951313,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.918331,0.990501,0.923723,NaN,NaN,0.0,0.0,NaN
3,446434757,00062442CB01,113492373,2025,Retro,ANDREW GOLDFARB MD PC,NaN,NaN,New York,35004.0,NY-35004,"Nassau County-Suffolk County, NY",11040.0,0.0,0.00,1.14,62.00,63.14,NaN,NaN,1.443722,0.957265,NaN,NaN,1.009755,0.990324,NaN,NaN,1.153808,1.006071,1.010540,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1.150568,1.017416,1.021846,NaN,NaN,0.0,0.0,NaN
4,3971678558,0006DCCB459E,742797745,2025,Retro,WELLMED MEDICAL MANAGEMENT OF FLORIDA INC,WellMed Tampa/Orlando Medicare ACO LLC,NaN,Florida,38940.0,FL-38940,"Port St. Lucie, FL",34994.0,40.0,238.00,184.50,7084.50,7547.00,1.098046,1.102582,1.051921,1.158040,1.098046,0.996534,0.968263,1.095377,1.084523,1.140871,1.256135,1.197251,1.192745,0.0,0.0,0.0,0.0,NaN,1.084523,1.116884,1.235580,1.125186,1.128155,0.0,0.0,0.0,0.0,NaN


select * from raw.milliman_ret_risk_20250522


In [7]:
for q in querries:
    print(q)

select * from raw.milliman_pro_risk_20250522
select * from raw.milliman_ret_risk_20250522
